# Embedded FXE Example with Blinky

Demonstrating the embedded FXE engine using nRF52832 blinky firmware example.
An elf file is used for better understanding of how the engine covers code, but can be done just as well with a binary file.

In [1]:
import os
import re
import shutil
import subprocess
from copy import deepcopy

from efxe import *

In [2]:
%%time
efxe = EmFXEngine(
    pd="mmaps/nrf52832.yml",
    fw_path="examples/blinky.elf",
    log_stdout=True,
    log_insn=True,
)
efxe.run()

2022-11-19 19:59:59,436 : backup context @ 0x2b1
2022-11-19 19:59:59,437 : backup context @ 0x2b1
2022-11-19 19:59:59,438 : backup context @ 0x2b1
2022-11-19 19:59:59,438 : backup context @ 0x2b1
2022-11-19 19:59:59,438 : backup context @ 0x2b1
2022-11-19 19:59:59,439 : backup context @ 0x2b1
2022-11-19 19:59:59,439 : backup context @ 0x2b1
2022-11-19 19:59:59,439 : backup context @ 0x2b1
2022-11-19 19:59:59,439 : backup context @ 0x2b1
2022-11-19 19:59:59,440 : backup context @ 0x2b1
2022-11-19 19:59:59,440 : backup context @ 0x2b1
2022-11-19 19:59:59,440 : backup context @ 0x2b1
2022-11-19 19:59:59,441 : backup context @ 0x2b1
2022-11-19 19:59:59,441 : backup context @ 0x2b1
2022-11-19 19:59:59,441 : backup context @ 0x2b1
2022-11-19 19:59:59,441 : backup context @ 0x2b1
2022-11-19 19:59:59,442 : backup context @ 0x2b1
2022-11-19 19:59:59,442 : backup context @ 0x2b1
2022-11-19 19:59:59,442 : backup context @ 0x2b1
2022-11-19 19:59:59,442 : backup context @ 0x2b1
2022-11-19 19:59:59,

UcError: Invalid memory write (UC_ERR_WRITE_UNMAPPED)

In [3]:
cs = Cs(CS_ARCH_ARM, CS_MODE_THUMB)
cs.detail = 1
total_insns = 0
total_size = 0
for block in efxe.bblocks.values():
    total_size += block.size
    for insn in cs.disasm(efxe.uc.mem_read(block.addr, block.size), block.size):
        total_insns += 1
print(f"Found {len(efxe.bblocks)} blocks, {total_insns} insns, {hex(total_size)} bytes\n")
# for k, v in efxe.bblocks.items():
#     print(hex(k), v)

Found 172 blocks, 687 insns, 0x77c bytes



## Code Coverage

The following code highlights any instructions in a printout of the disassembly that were in a found basic block.

In [4]:
disasm_txt = deepcopy(efxe.fw.disasm_txt)
for block in efxe.bblocks.values():
    insn_addr = block.addr
    for insn in efxe.cs.disasm(
            efxe.uc.mem_read(block.addr, block.size), block.size):
        try:
            lineno = efxe.fw.disasm[insn_addr]['line']
            insn_addr += insn.size
            disasm_txt[lineno] = "{}{}{}{:<100}\u001b[0m".format(
                "\u001b[103m",  # bright yellow background
                "\u001b[30m",   # black foreground
                "\u001b[1m",    # bold font
                disasm_txt[lineno])
        except Exception:
            pass
print('\n'.join(disasm_txt))


examples/blinky.elf:     file format elf32-littlearm


Disassembly of section .text:

00000000 <__isr_vector>:
   0:	20010000 	.word	0x20010000
   4:	000002b1 	.word	0x000002b1
   8:	000002d9 	.word	0x000002d9
   c:	000002db 	.word	0x000002db
  10:	000002dd 	.word	0x000002dd
  14:	000002df 	.word	0x000002df
  18:	000002e1 	.word	0x000002e1
	...
  2c:	000002e3 	.word	0x000002e3
  30:	000002e5 	.word	0x000002e5
  34:	00000000 	.word	0x00000000
  38:	000002e7 	.word	0x000002e7
  3c:	000002e9 	.word	0x000002e9
  40:	000002eb 	.word	0x000002eb
  44:	000002eb 	.word	0x000002eb
  48:	000002eb 	.word	0x000002eb
  4c:	000002eb 	.word	0x000002eb
  50:	000002eb 	.word	0x000002eb
  54:	000002eb 	.word	0x000002eb
  58:	000002eb 	.word	0x000002eb
  5c:	000002eb 	.word	0x000002eb
  60:	000002eb 	.word	0x000002eb
  64:	000002eb 	.word	0x000002eb
  68:	000002eb 	.word	0x000002eb
  6c:	000002eb 	.word	0x000002eb
  70:	000002eb 	.word	0x000002eb
  74:	000002eb 	.word	0x000002eb
  78:	000002eb 	.word	0x0